In [ ]:
1. python train.py
2. python test.py

test 폴더에 바꾸고 싶은 노래 넣기
config.py의 direction = "B2A"는 B를 A로 바꿈.

결과는 sample폴더 밑에 생김

In [2]:
!pip install pyworld

  Using cached https://files.pythonhosted.org/packages/dc/8d/f83aef293df2fb9f3435f129ea7a02f55fd0fe04ada69bf4207d5ffbc92c/pyworld-0.2.8.tar.gz
  Created wheel for pyworld: filename=pyworld-0.2.8-cp35-cp35m-win_amd64.whl size=221958 sha256=80ed458a1f29a64c9145e8063227d99b078ce0e946293c3bd2f06f0cc9df4884
  Stored in directory: C:\Users\Kim Yuum\AppData\Local\pip\Cache\wheels\c3\58\e5\a7e39ab92c56825f976970b97066753c68406c7fb0d80d4a4a
Successfully built pyworld


In [ ]:
%run train.py

Constructing MCEPs....


In [ ]:
import tensorflow as tf
import numpy as np
import os, time

from cyclegan import CycleGAN
from cycle_began import CycleBeGAN
from preprocess import *
import pickle

n_epochs = 1
mini_batch_size = 1  # mini_batch_size = 1 is better
generator_learning_rate = 0.0002
generator_learning_rate_decay = generator_learning_rate / 200000
discriminator_learning_rate = 0.0001
discriminator_learning_rate_decay = discriminator_learning_rate / 200000
sampling_rate = 16000
num_mcep = 24
frame_period = 5
## n_frames 128 is about 0.5 sec
n_frames = 512
lambda_cycle = 10
lambda_identity = 5
# 추가
gamma_A = 0.5
gamma_B = 0.5
lambda_k_A = 0.001
lambda_k_B = 0.001
balance_A = 0
balance_B = 0
# kta 초기값
k_t_A = 0
k_t_B = 0

with open('A.txt', 'rb') as f:
    A_norm = pickle.load(f)

with open('B.txt', 'rb') as k:
    B_norm = pickle.load(k)

model = CycleBeGAN(num_features = n_features, log_dir = log_dir)

print("Start CycleBeGan Training...")
for epoch in range(n_epochs) :
    print("Epoch : %d " % epoch ) 
    start_time = time.time()
    train_A, train_B = sample_train_data(dataset_A=A_norm, dataset_B=B_norm,n_frames=n_frames) # random data

    n_samples = train_A.shape[0]

    for i in range(n_samples) : # mini_ batch_size = 1
        n_iter = n_samples * epoch + i
        if n_iter % 50 == 0:
            k_t_A = k_t_A + (lambda_k_A *balance_A)
            if k_t_A > 1:
                k_t_A = 1
            if k_t_A <= 0 :
                k_t_A = 0
            
            k_t_B = k_t_B + (lambda_k_B *balance_B)
            if k_t_B > 1:
                k_t_B = 1
            if k_t_B <= 0:
                k_t_B = 0
    
        if n_iter > 10000 :
            identity_lambda = 0
        if n_iter > 200000 :
            generator_lr = max(0, generator_lr - generator_lr_decay)
            discriminator_lr = max(0, discriminator_lr - discriminator_lr_decay)

        start = i
        end = start + 1
        generator_loss, discriminator_loss, measure_A, measure_B, k_t_A, k_t_B, balance_A, balance_B = model.train(
#                         input_A=dataset_A[start:end], input_B=dataset_B[start:end], 
                        input_A=train_A[start:end], input_B=train_B[start:end],
                        lambda_cycle=lambda_cycle,
                        lambda_identity=lambda_identity,
                        gamma_A=gamma_A, gamma_B=gamma_B, lambda_k_A=lambda_k_A, lambda_k_B=lambda_k_B,
                        generator_learning_rate=generator_learning_rate,
                        discriminator_learning_rate=discriminator_learning_rate, 
                        k_t_A = k_t_A, k_t_B = k_t_B)
    end_time = time.time()
    epoch_time = end_time-start_time
    print("Generator Loss : %f, Discriminator Loss : %f, Time : %02d:%02d" % (generator_loss, discriminator_loss,(epoch_time % 3600 // 60),(epoch_time % 60 // 1)))
    model.save(directory = model_dir, filename = "Cycle_BeGan")

W1225 23:47:12.588104  2964 module_wrapper.py:139] From C:\Users\Kim Yuum\Desktop\TOBIGS2\Project\sst\cycle_began.py:37: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1225 23:47:12.634953  2964 module_wrapper.py:139] From C:\Users\Kim Yuum\Desktop\TOBIGS2\Project\sst\Utils\networks.py:104: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1225 23:47:12.634953  2964 deprecation.py:323] From C:\Users\Kim Yuum\Desktop\TOBIGS2\Project\sst\Utils\layers.py:233: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
W1225 23:47:12.657838  2964 deprecation.py:323] From C:\Users\Kim Yuum\AppData\Local\conda\conda\envs\yum_env\lib\site-packages\tensorflow_core\python\layers\convolutional.py:218: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a futur

Start CycleBeGan Training...
Epoch : 0 


In [ ]:
%run train.py

Constructing MCEPs....
